## 4 -  Usando o modelo original do notebook:

a. Mude o Otimizador: Substitua o otimizador Adam por SGD (Stochastic Gradient Descent). 

b. Treine o modelo com o SGD.

c. O que aconteceu com o custo (loss) durante o treinamento? A acurácia final foi melhor ou pior? O SGD com essa taxa de aprendizado pareceu uma boa escolha?

O custo aumentou um pouco durante o treinamento. Antes da alteração, ele se mantinha constante em cerca de 0,693, mas com a mudança na taxa de aprendizado, passou a variar e atingir valores maiores em até aproximadamente 0,2. Com a modificação da learning rate, houve uma melhora na acurácia do modelo que utiliza SGD. No entanto, com a taxa de aprendizado anterior, esse modelo não foi uma boa escolha, pois o custo aumentava e a acurácia final era similar à obtida com o otimizador Adam.

In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F


In [26]:
import numpy as np
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
titanic = sns.load_dataset('titanic')

feature_names = ['pclass', 'female', 'age', 'fare']
titanic['female'] = titanic['sex'].map({'male': 0, 'female': 1})
titanic.dropna(subset=feature_names, inplace=True)  #891 para 714

X = titanic[feature_names].to_numpy()
y = titanic['survived'].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.25,
                                                    random_state=123)

In [27]:
print('Tamanho de X_train: ', X_train.shape)
print('Tamanho de X_test: ', X_test.shape)
print('Tamanho de y_train: ', y_train.shape)
print('Tamanho de y_test: ', y_test.shape)

Tamanho de X_train:  (535, 4)
Tamanho de X_test:  (179, 4)
Tamanho de y_train:  (535,)
Tamanho de y_test:  (179,)


In [28]:
class ClassBin(nn.Module):
    # Construtor
    def __init__(self):
        super(ClassBin, self).__init__()
        self.linear1 = nn.Linear(4, 4)    # primeira hidden layer
        self.dropout1 = nn.Dropout(0.2)   # dropout layer
        self.linear2 = nn.Linear(4, 4)    # segunda hidden layer
        self.dropout2 = nn.Dropout(0.2)   # dropout layer
        self.linear3 = nn.Linear(4, 1)    # terceira hidden layer
        self.dropout3 = nn.Dropout(0.2)   # dropout layer
        self.sigmoid = nn.Sigmoid()

    # Propagação (Feed Forward)
    def forward(self, x):
        x = F.relu(self.linear1(x))
        x = self.dropout1(x)
        x = F.relu(self.linear2(x))
        x = self.dropout2(x)
        x = F.relu(self.linear3(x))
        x = self.dropout3(x)
        x = self.sigmoid(x)
        return x

model = ClassBin()
print(model)

ClassBin(
  (linear1): Linear(in_features=4, out_features=4, bias=True)
  (dropout1): Dropout(p=0.2, inplace=False)
  (linear2): Linear(in_features=4, out_features=4, bias=True)
  (dropout2): Dropout(p=0.2, inplace=False)
  (linear3): Linear(in_features=4, out_features=1, bias=True)
  (dropout3): Dropout(p=0.2, inplace=False)
  (sigmoid): Sigmoid()
)


In [29]:
loss_fn = nn.BCELoss()
epochs = 100
batch_size = 32  # X_train 535 / 32 = 16.71 (então são 17 batches de 32)
learning_rate = 0.001

# Instânciar o Otimizador Adam
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [30]:
from torch.utils.data import TensorDataset, DataLoader

# Converter X e y para torch.Tensor
X_train = torch.Tensor(X_train)
y_train = torch.Tensor(y_train)
X_test = torch.Tensor(X_test)
y_test = torch.Tensor(y_test)

# Um Dataset de Tensores - Array [X, y]
train = TensorDataset(X_train, y_train)
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)

test = TensorDataset(X_test, y_test)
test_loader = DataLoader(test, batch_size=batch_size, shuffle=True)

In [31]:
for t in range(epochs):
    model.train() # Colocar o modelo em modo de treinamento (calcula os gradientes)
    
    # Batch Size
    for data in train_loader:
        # dar nome aos bois
        X = data[0]
        y = data[1]
        
        # Propagação (Feed Forward)
        y_pred = model(X)
    
        # Calcular erro usando a função-custo
        # y precisa virar um Tensor com tamanho (batch_size, 1)
        loss = loss_fn(y_pred, y.unsqueeze_(1))
        
        # Zera os gradientes antes da Retro-propagação (Backpropagation)
        model.zero_grad()

        # Retro-propagação (Backpropagation)
        loss.backward()

        # Atualização dos parâmetros
        optimizer.step()

    # Fim da Época
    print(f"""Época {t + 1},
          Custo Treino: {round(loss.item(), 3)}""")

Época 1,
          Custo Treino: 0.707
Época 2,
          Custo Treino: 0.69
Época 3,
          Custo Treino: 0.692
Época 4,
          Custo Treino: 0.709
Época 5,
          Custo Treino: 0.703
Época 6,
          Custo Treino: 0.693
Época 7,
          Custo Treino: 0.69
Época 8,
          Custo Treino: 0.696
Época 9,
          Custo Treino: 0.696
Época 10,
          Custo Treino: 0.691
Época 11,
          Custo Treino: 0.7
Época 12,
          Custo Treino: 0.7
Época 13,
          Custo Treino: 0.701
Época 14,
          Custo Treino: 0.682
Época 15,
          Custo Treino: 0.69
Época 16,
          Custo Treino: 0.693
Época 17,
          Custo Treino: 0.696
Época 18,
          Custo Treino: 0.695
Época 19,
          Custo Treino: 0.691
Época 20,
          Custo Treino: 0.688
Época 21,
          Custo Treino: 0.685
Época 22,
          Custo Treino: 0.69
Época 23,
          Custo Treino: 0.7
Época 24,
          Custo Treino: 0.695
Época 25,
          Custo Treino: 0.679
Época 26,
         

In [32]:
model.eval() # coloca o modelo em modo de avaliação (sem calcular gradientes)

train_pred = model(X_train)
train_pred = train_pred.detach().apply_(lambda x : 1 if x > 0.5 else 0)
train_acc = torch.sum(train_pred.flatten() == y_train) / train_pred.size(0)

test_pred = model(X_test)
test_pred = test_pred.detach().apply_(lambda x : 1 if x > 0.5 else 0)
test_acc = torch.sum(test_pred.flatten() == y_test) / test_pred.size(0)

print(f"Acurácia de Treino: {train_acc}")
print('\n ---------------------------\n')
print(f"Acurácia de Teste: {test_acc}")

Acurácia de Treino: 0.6000000238418579

 ---------------------------

Acurácia de Teste: 0.6033519506454468
